# Model Evaluation and analysis template

You can follow this python script to conduct model evaluation and analysis.

## Import necessary python packages and import the prewritten functions.

In [ ]:
import micropip
await micropip.install("distutils")
await micropip.install("plotly==5.13.0")
await micropip.install("nbformat==5.7.3")

In [ ]:
%pip install -q nbformat plotly statsmodels

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os

# import hvplot.xarray
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# import cartopy 
# import cartopy.crs as ccrs
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

from functions import aerosol_dehm

# Part I: Model Evaluation

## 1. Load the data

We have weekly data of simulated and observed SO2 (ug/m3), SO4 (ugS/m3), NH4 (ugN/m3), NO3 (ugN/m3), and PM2.5 (ug/m3) at three stations: Villum, Spitzbergen, and Alert.

In [3]:
# We totally have the following three stations available:
allstations = ['Villum','Spitzbergen','Alert']

# A function to get the data of one station
def get_station_data(station):
    input_file = f'data/Arctic_aerosol_{station}.csv'

    df = pd.read_csv(input_file,header=0)
    # Convert date column to datetime format
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')

    return df

In [4]:
station = 'Alert'
df = get_station_data(station)
print(df.head(5))

            SO2_mod  SO2_mea  SO4_mod  SO4_mea  NH4_mod  NH4_mea  NO3_mod  \
date                                                                        
2007-01-01    0.405      NaN    0.080    0.288    0.024    0.072    0.037   
2007-01-08    1.700      NaN    0.241      NaN    0.045      NaN    0.070   
2007-01-15    0.973      NaN    0.250    0.208    0.037    0.033    0.051   
2007-01-22    0.330      NaN    0.115    0.230    0.034    0.044    0.058   
2007-01-29    0.408      NaN    0.116    0.253    0.036    0.056    0.043   

            NO3_mea  PM2.5_mod  
date                            
2007-01-01    0.026      1.510  
2007-01-08      NaN      2.416  
2007-01-15    0.018      3.583  
2007-01-22    0.027      1.545  
2007-01-29    0.048      4.677  


## 2. Plot the time series

In [5]:
# Function to plot the variable

def plot_time_series(var,station,df=None):
    df = get_station_data(station) if df is None else df

    fig = go.Figure()
    # Add simulation line
    fig.add_trace(go.Scatter(y=df[f'{var}_mod'],x=df.index,name='model simulation'))

    # Add observation line
    if f'{var}_mea' in df.columns:
        fig.add_trace(go.Scatter(y=df[f'{var}_mea'],x=df.index,name='measurement'))
    
    # Title and legend
    fig.update_layout(title = f'{var} time series at {station}')
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
        ))
    fig.update_layout(showlegend=True) 
    fig.show()


In [7]:
var,station = 'SO4','Villum'   # Choose from SO2, SO4, NH4, NO3, and PM2.5
plot_time_series(var,station)

Now let's look at annual mean

In [8]:
# Look at annual mean
annual_df = df.groupby(pd.Grouper(freq='1Y')).mean()

plot_time_series(var,station,df=annual_df)


Seasonal cycle

In [9]:
df['date'] = pd.to_datetime(df.index)
seasonal_df = df.groupby(df.date.dt.month).mean()

plot_time_series(var,station,df=seasonal_df)

## 3. 1-1 scatter plot

In [17]:
def plot_scatter(var,station,df=None):
    df = get_station_data(station) if df is None else df
    if f'{var}_mea' not in df.columns:
        print(f'No measurement of {var} available')
        return
    
    vmin,vmax = 0,max(df[f'{var}_mea'].max(),df[f'{var}_mod'].max())*1.02
    fig = px.scatter(df, x=f'{var}_mea', y=f'{var}_mod', trendline="ols",width=800,height=800)
    fig.update_layout(title = f'{var} measurement vs model simulation at {station}')
    fig.update_layout(yaxis=dict(range=[vmin,vmax]))
    fig.update_layout(xaxis=dict(range=[vmin,vmax]))
    fig.show()

    results = px.get_trendline_results(fig)
    print(results.px_fit_results.iloc[0].summary())


In [18]:
var,station = 'NH4','Villum'
plot_scatter(var,station)


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.419
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     271.6
Date:                Sun, 05 Feb 2023   Prob (F-statistic):           2.58e-46
Time:                        13:20:54   Log-Likelihood:                 985.70
No. Observations:                 378   AIC:                            -1967.
Df Residuals:                     376   BIC:                            -1960.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0206      0.001     16.013      0.0

# Part II. DEHM (Danish Eulerian Hemispheric Model) field output visualization

In [19]:
dehm = aerosol_dehm()
ds = dehm.ds.load()
print(ds)

<xarray.Dataset>
Dimensions:      (x: 300, y: 300)
Coordinates:
  * x            (x) int64 0 1 2 3 4 5 6 7 8 ... 292 293 294 295 296 297 298 299
  * y            (y) int64 0 1 2 3 4 5 6 7 8 ... 292 293 294 295 296 297 298 299
    z            int64 0
Data variables:
    SO2_ugSm-3   (x, y) float64 0.4712 0.4702 0.4705 ... 0.1892 0.189 0.189
    NH4_ugNm-3   (x, y) float64 1.692 1.697 1.697 1.333 ... 0.5821 0.5822 0.5821
    SO4_ugSm-3   (x, y) float64 0.4039 0.4038 0.4037 ... 0.493 0.4929 0.4928
    tSO4_ugSm-3  (x, y) float64 0.6997 0.699 0.699 ... 0.5609 0.5609 0.5607
    PM2.5_ugm-3  (x, y) float64 9.284 9.293 9.301 7.807 ... 4.516 4.517 4.516
    PM10_ugm-3   (x, y) float64 9.927 9.922 9.922 8.229 ... 4.843 4.843 4.842
    SIA_ugm-3    (x, y) float64 6.193 6.2 6.199 5.138 ... 3.056 3.201 3.201 3.2
    lat          (x, y) float64 40.78 40.92 41.06 41.2 ... 43.63 43.48 43.34
    lon          (x, y) float64 -77.0 -77.19 -77.37 -77.56 ... 102.6 102.8 103.0


In [ ]:
%matplotlib inline
fig = dehm.plot_annual_map(vmin=0,vmax=1)
fig.show()

In [ ]:
%matplotlib inline
fig = dehm.plot_annual_map(par='PM2.5_ugm-3',vmin=0,vmax=10)
# fig = dehm.plot_annual_map(vmin=0,vmax=1)
fig.show()

In [22]:
import plotly.graph_objects as go

var = 'PM2.5_ugm-3'
dehm = aerosol_dehm()
df = dehm.map_to_dataframe(var=var)
# print(df['lat'])

# fig = px.density_mapbox(df, lat='lat', lon='lon', z=var, radius=20,
#                         center=dict(lat=90, lon=-30), zoom=1,
#                         mapbox_style="stamen-terrain")
fig = go.Figure(
    data =
    go.Contour(
        z=df[var],
        # x=df['x'], # horizontal axis
        # y=df['y'] # vertical axis
    ))
fig.show()
# fig.show()

# fig = go.Figure(go.Scattergeo())
# fig.update_geos(projection_type="mercator")
# fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
# # fig.add_trace(go.Contour(z=df[var],x=df['lon'],y=df['lat'], showscale=False, connectgaps=True))
# fig.show()